In [15]:
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import dash_ag_grid as dag


num_rows = 10000

# Создаем примеры данных
data = {
    'Дата': pd.date_range(start='2023-01-01', periods=num_rows, freq='h'),
    'Контрагент': np.random.choice(['Компания А', 'Компания B', 'Компания C', 'Компания D', 
                                    'Компания E', 'Компания F', 'Компания G', 'Компания H', 
                                    'Компания I', 'Компания J'], size=num_rows),
    'Сумма': np.random.randint(-1000, 50000, size=num_rows),
    'Договор': [f'Договор №{i}' for i in np.random.randint(1, 1000, size=num_rows)],
    'Статья учета': np.random.choice(['Услуги', 'Товары', 'Аренда', 'Зарплата', 'Командировки', 
                                      'Закупки', 'Прочее', 'Капитальные затраты', 'Реклама', 
                                      'Консультации'], size=num_rows),
    'Банковский счет': [f'Счет №{np.random.randint(1000, 9999)}' for _ in range(num_rows)]
}

df = pd.DataFrame(data)
df = df.head()
dataTypeDefinitions = {
    "object": {
        "baseDataType": "object",
        "extendsDataType": "object",
        "valueParser": {"function": "({ name: params.newValue })"},
        "valueFormatter": {"function": "params.value == null ? '' : params.value.name"},
    },
}

col_defs = []
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        col_type = 'Number'
    elif pd.api.types.is_datetime64_any_dtype(df[col]):
        col_type = 'dateString'
    else:
        col_type = 'text'
    col_defs.append({"field": col, 'editable': True})

datatable = dag.AgGrid(id='datatable',
                       rowData=df.to_dict('records'),
                       columnDefs=col_defs,
                       dashGridOptions={"dataTypeDefinitions": dataTypeDefinitions, "animateRows": False},
                       suppressDragLeaveHidesColumns=False,
                       persistence=True,
                       defaultColDef={'filter': True,
                                      'floatingFilter': True,
                                      'resizable': True,
                                      'sortable': True,
                                      'editable': False,
                                      'minWidth': 125,
                                      'wrapHeaderText': True,
                                      'autoHeaderHeight': True},
                       style={'resize': 'both', 'overflow': 'hidden'}
                       )
fig_contract = go.Figure()
fig_contract.add_trace(
    go.Bar(
            x=df['Статья учета'], 
            y=df['Сумма'], 
            name='Статья'))
fig_contract.add_trace(go.Bar(
        x=df['Контрагент'], 
        y=df['Сумма'], 
        name='Контрагент'))
fig_contract.update_layout(
        title=' ', 
        barmode='group')
    
datatable  

AgGrid(id='datatable', columnDefs=[{'field': 'Дата', 'editable': True}, {'field': 'Контрагент', 'editable': True}, {'field': 'Сумма', 'editable': True}, {'field': 'Договор', 'editable': True}, {'field': 'Статья учета', 'editable': True}, {'field': 'Банковский счет', 'editable': True}], dashGridOptions={'dataTypeDefinitions': {'object': {'baseDataType': 'object', 'extendsDataType': 'object', 'valueParser': {'function': '({ name: params.newValue })'}, 'valueFormatter': {'function': "params.value == null ? '' : params.value.name"}}}, 'animateRows': False}, defaultColDef={'filter': True, 'floatingFilter': True, 'resizable': True, 'sortable': True, 'editable': False, 'minWidth': 125, 'wrapHeaderText': True, 'autoHeaderHeight': True}, persistence=True, rowData=[{'Дата': Timestamp('2023-01-01 00:00:00'), 'Контрагент': 'Компания H', 'Сумма': 3862, 'Договор': 'Договор №597', 'Статья учета': 'Командировки', 'Банковский счет': 'Счет №8427'}, {'Дата': Timestamp('2023-01-01 01:00:00'), 'Контрагент'